In [ ]:
!git clone https://github.com/jaywalnut310/vits.git

fatal: destination path 'vits' already exists and is not an empty directory.


In [ ]:
!cd /content/vits

In [ ]:
!pip install -r /content/vits/requirements.txt
!pip install Unidecode==1.1.1

In [ ]:
%cd /content/vits  # If you restart runtime
%cd /content/vits/monotonic_align

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
!sudo apt-get install espeak -y

In [ ]:
!cd ..

In [ ]:
!pip install gdown

In [ ]:
%cd /content/vits
!gdown 'https://drive.google.com/uc?id=1q86w74Ygw2hNzYP9cWkeClGT5X25PvBT'

In [ ]:
%cd /content/vits

In [ ]:
# !cd /content/vits/monotonic_align
# !python /content/vits/monotonic_align/setup.py build_ext --inplace

In [ ]:
!pip install phonemizer

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm).to(device)
    return text_norm

In [ ]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [ ]:
%cd /content/vits

## Multi Speaker

In [ ]:
!gdown 'https://drive.google.com/uc?id=11aHOlhnxzjpdWDpsz1vFDCzbeEfoIxru'

In [ ]:
hps_ms = utils.get_hparams_from_file("./configs/vctk_base.json")

In [ ]:
net_g_ms = SynthesizerTrn(
    len(symbols),
    hps_ms.data.filter_length // 2 + 1,
    hps_ms.train.segment_size // hps.data.hop_length,
    n_speakers=hps_ms.data.n_speakers,
    **hps_ms.model)
# _ = net_g.eval()

_ = utils.load_checkpoint("pretrained_vctk.pth", net_g_ms, None)

In [ ]:
import torchaudio
import os
from tqdm import tqdm
import re
from vits.text import text_to_sequence

In [ ]:
  import os
  import torchaudio
  from tqdm import tqdm

# Set the device to use for inference
  device = "cuda" if torch.cuda.is_available() else "cpu"

# Define a function to handle the error and skip files
    # def handle_error(file_path):
    #   print(f"{file_path} has been skipped.")

# Loop through all the text files in the directory
# for file_name in os.listdir('/content'):
#   if file_name.endswith('.txt'):
#     # Open the file
  with open("/content/concatenated_sentences.txt", "r") as f:
    lines = f.readlines()
    
  id = 0 # Initialize speaker identity
  i = 1;
    
  for line in tqdm(lines):
    if i >= 53762: ###### Input the file numbers to iterate through
      if i < 55000:
        try:
            line = line.strip()
            
            # Extract the number and sentence from the line
            num, sentence = line.split(" ", 1)
            stn_tst = get_text(sentence, hps_ms)
            # print(sentence)
            # Check if the speaker identity needs to be updated
            if i % 20 == 1:
                id += 1
                id = id % 108 # There are 82 speakers
            
            # Generate the audio for the sentence
            with torch.no_grad():
                # Prepare the input tensors
                x_tst = stn_tst.unsqueeze(0)
                x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
                sid = torch.LongTensor([id]).to(device)

                # Move the tensors to the device
                net_g_ms = net_g_ms.to(device)
                x_tst = x_tst.to(device)
                # print(x_tst)
                x_tst_lengths = x_tst_lengths.to(device)
                # print(x_tst_lengths)
                audio = net_g_ms.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0, 0].data.float().cpu().numpy()

                file_name = f"{num}.wav"
                num_channels = 1  # mono audio
                num_samples = len(audio)
                audio_tensor = torch.from_numpy(audio).view(num_channels, num_samples)  # convert to 2D tensor
                folder = "/content/audio"
                if not os.path.exists(folder):
                    os.makedirs(folder)
                file_path = os.path.join(folder, file_name)
                torchaudio.save(file_path, audio_tensor, hps_ms.data.sampling_rate)
            
            i += 1
            
        except:
            print(f"{num} has been skipped")
            i += 1
            
    else:
        i += 1


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

In [ ]:
!gsutil -m cp -r /content/audio gs://idl_project1/Train-Clean-360/

In [ ]:
from google.colab import files

files.download('/content/audio/1472-139797-0039.wav')

In [ ]:
  # !zip -r /content/audio_zip.zip /content/audio

In [ ]:
!curl https://sdk.cloud.google.com | bash
!exec -l $SHELL
!gcloud init

In [ ]:
!gsutil -m cp -r /content/audio gs://idl_project1/


In [ ]:
# from tqdm import tqdm

# Set the device to use for inference
device = "cuda" if torch.cuda.is_available() else "cpu"

with open("/content/combined_text.txt", "r") as f:
    lines = f.readlines()
    

id = 0 # Initialize speaker identity
i = 1;
for line in tqdm(lines):
  if i >=7167:
    if i<15000:
      line = line.strip()
      
      # Extract the number and sentence from the line
      # num, sentence = line.split(" ", 1)
      # stn_tst = get_text(sentence, hps_ms)
      # num = int(num)
      print(num)
      print(sentence)
      # Check if the speaker identity needs to be updated
      if i % 20 == 1:
        id += 1
        id = id % 108 # There are 82 speakers
        i+=1;
    # Generate the audio for the sentence
    
    # sid = torch.LongTensor([id]) # speaker identity
    # sid = sid.cuda()

    # stn_tst = get_text("Can I test this model with different people please", hps_ms)

      with torch.no_grad():
        # Prepare the input tensors
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        sid = torch.LongTensor([id]).to(device)

        # Move the tensors to the device
        net_g_ms = net_g_ms.to(device)
        x_tst = x_tst.to(device)
        x_tst_lengths = x_tst_lengths.to(device)
        audio = net_g_ms.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0, 0].data.float().cpu().numpy()

        file_name = f"{num}.wav"
        num_channels = 1  # mono audio
        num_samples = len(audio)
        audio_tensor = torch.from_numpy(audio).view(num_channels, num_samples)  # convert to 2D tensor
        folder = "/content/audio"
        if not os.path.exists(folder):
            os.makedirs(folder)
        file_path = os.path.join(folder, file_name)
        torchaudio.save(file_path, audio_tensor, hps_ms.data.sampling_rate)
    else:
      i+=1;
  else:
    i +=1;

In [ ]:
!gcloud auth login

In [ ]:
!gcloud config set project 

In [ ]:
!gcloud config set project verdant-wave-384615

## Voice Conversion

In [ ]:
!wget https://jaywalnut310.github.io/vits-demo/wavs/vc_01_01.wav

In [ ]:
from mel_processing import spectrogram_torch
from utils import load_wav_to_torch

audio, sampling_rate = load_wav_to_torch("./vc_01_01.wav")
        
y = audio / hps_ms.data.max_wav_value
y = y.unsqueeze(0)

spec = spectrogram_torch(y, hps_ms.data.filter_length,
    hps_ms.data.sampling_rate, hps_ms.data.hop_length, hps_ms.data.win_length,
    center=False)
spec_lengths = torch.LongTensor([spec.size(-1)])
sid_src = torch.LongTensor([81])

In [ ]:
with torch.no_grad():
    sid_tgt1 = torch.LongTensor([77])
    sid_tgt2 = torch.LongTensor([14])
    sid_tgt3 = torch.LongTensor([1])
    sid_tgt4 = torch.LongTensor([17])
    audio1 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.float().numpy()
    audio2 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.float().numpy()
    audio3 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.float().numpy()
    audio4 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt4)[0][0,0].data.float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].numpy(), rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt4.item())
ipd.display(ipd.Audio(audio4, rate=hps_ms.data.sampling_rate))